<a href="https://colab.research.google.com/github/Gilberto-Calixto/MentorIA/blob/main/Meu_Mentor_de_Estudos_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")




In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

model = "gemini-2.0-flash"

In [ ]:
!pip install -q google-adk

In [ ]:
# Dependências para criar r formatar agentes de ia
from google.adk.agents import Agent

from google.adk.runners import Runner

from google.adk.sessions import InMemorySessionService

from google.adk.tools import google_search

from google.genai import types # Para criar conteúdos (Content e Part)

from datetime import date

import textwrap # Para formatar melhor a saída de texto

from IPython.display import display, Markdown # Para exibir texto formatado no Colab

import requests # Para fazer requisições HTTP

import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Criador de agente

#session_service = InMemorySessionService()

def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()

    # Criar uma nova sessão (você pode personalizar os IDs conforme o necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

    # Criar Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    # Criar conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""

    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"

    return final_response


In [ ]:
# Criar Agente 1 -> Gerador de Resumos

def agente_resumidor(assunto, data_de_hoje):
    resumidor = Agent(
        name="agente_resumidor",
        model="gemini-2.0-flash",
        instruction='''
        Você é um assistente especialista em criar resumos de estudos, desde pequenos a grandes textos.
        A sua tarefa é usar a ferramenta de busca do Google (google_search).

        Busque conhecimento e soluções relevantes, bem descritas e por ordem de relevância no aprendizado
        sobre o conteúdo passado.

        Priorize até 5 conteúdos dos mais relevantes sobre o assunto passado, levando em conta feedbacks,
        avaliação e visibilidade.

        Ignore conteúdos com pouca relevância e visibilidade, a não ser que sejam as únicas opções.

        Busque prioritariamente os conteúdos e assuntos mais atuais, de no máximo um ano em relação à data de hoje.
        Caso eles não atendam aos requisitos passados, busque outras fontes.
        ''',
        description="Agente que resume busca de conteúdos de estudo",
        tools=[google_search]
    )

    entrada_do_agente_resumidor = f"Assunto: {assunto}\nData de hoje: {data_de_hoje}"
    resumo = call_agent(resumidor, entrada_do_agente_resumidor)

    return resumo


In [ ]:
# Agente Criador de Quiz

def agente_criador_quiz(assunto, qtd):

  quiz_agent = Agent(
      name="quiz_agent",
      model= "gemini-2.0-flash",
      instruction=f'''
      Você é um asistente de estudos que tem a responsabilidade e função de criar
      quizzes de estudo. baseados no {assunto} informado(caso o assunto venham
      com muito conteúdo em excesso, resuma, mas resuma bem pouco) e a quantidade
      informada {qtd}.
      Siga a ordem de pergunta e respostas do modelo de quiz.

      a cada quiz informe se acertou ou errou.

      no final liste as perguntas e respostas com os acertos e erros.
      ''',

      description= "Agente que cria quizzes de estudo",
  )

  entrada = f" Assunto: {assunto}\nQuantidade de perguntas do quiz: {qtd}"
  result = call_agent(quiz_agent, entrada)
  return result

In [ ]:
# Agente Explicador de Conceitos:

def agente_explicador_de_conceitos(assunto):

  explicador_de_conceitos = Agent(
      name="explicador_de_conceitos",
      model= "gemini-2.0-flash",
      instruction=f'''
      Você é especialista em explicar conceitos de forma clara e entendível com base
      no nível de quem você vai instruir.

      Seguindo o seguinte formato, explique os principais conceitos do assunto informado,
      de forma clara e objetiva. Com alegorias e exemplos.

      Lembre-se! explique da melhor forma possível e com uma dose de
      entusiasmos e criatividade.
      ''',
      description="Agente Explicador"
  )

  entrada_exp = f"Assunto: {assunto}"
  result_explicador = call_agent(explicador_de_conceitos, entrada_exp)
  return result_explicador

In [ ]:
date_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de MentorIA com 3 Agentes 🚀")

assunto = input("❓ Por favor, digite o ASSUNTO quer aprender e conhecer melhor: ")

if not assunto:

  print("Você esqueceu de digitar o assunto!")

else:

  print(f"Maravilha! Aprender mais sobre {assunto}")


  ######## AGENTE 1 ##########

  gerado = agente_resumidor(assunto=assunto, data_de_hoje=date_hoje)

  print("\n--- 📝 Resultado do Agente 1 (Resumidor) ---\n")

  display(gerado)

  print("--------------------------------------------------------------")

  ######## AGENTE 2 ##########
  quiz_result = agente_criador_quiz(assunto=assunto, qtd=10)
  print("\n--- 📝 Resultado do Agente 2 (Gerador de Quiz) ---\n")


  display(quiz_result)
  print("--------------------------------------------------------------")

  ######## AGENTE 3 ##########
  explicacao = agente_explicador_de_conceitos(assunto=assunto)
  print("\n--- 📝 Resultado do Agente 3 (Explicador) ---\n")


  display(explicacao)
  print("--------------------------------------------------------------")


🚀 Iniciando o Sistema de MentorIA com 3 Agentes 🚀
❓ Por favor, digite o ASSUNTO quer aprender e conhecer melhor: kotlin
Maravilha! Aprender mais sobre kotlin

--- 📝 Resultado do Agente 1 (Buscador) ---



'Para fornecer um resumo conciso e relevante sobre Kotlin, farei algumas buscas no Google para identificar os recursos mais atuais e informativos, priorizando aqueles com boa avaliação e visibilidade.\n\n\nCom base nas buscas realizadas, aqui está um resumo sobre Kotlin, priorizando informações atuais e relevantes para 2025:\n\n**Kotlin: Visão Geral**\n\nKotlin é uma linguagem de programação moderna, concisa e de tipagem estática, desenvolvida pela JetBrains (a mesma empresa por trás do IntelliJ IDEA). Ela é interoperável com Java, o que significa que você pode usar código Java existente em projetos Kotlin e vice-versa. Kotlin é a linguagem oficial do Google para desenvolvimento Android, mas também pode ser usada para desenvolvimento web, server-side e multi plataforma.\n\n**Principais Características e Vantagens:**\n\n*   **Concisão e Legibilidade:** Kotlin reduz a quantidade de código boilerplate, tornando o código mais fácil de ler e manter.\n*   **Segurança Nula (Null Safety):** Um

--------------------------------------------------------------

--- 📝 Resultado do Agente 1 (Buscador) ---



'Ok, vamos criar um quiz de 10 perguntas sobre Kotlin. Prepare-se para testar seus conhecimentos!\n\n**Quiz de Kotlin**\n\n1.  Qual palavra-chave é usada para declarar uma variável imutável em Kotlin?\n    *   a) `var`\n    *   b) `val`\n    *   c) `const`\n    *   d) `static`\n    *   **Sua resposta:**\n\n2.  Qual operador é usado para verificar igualdade estrutural em Kotlin?\n    *   a) `==`\n    *   b) `===`\n    *   c) `equals()`\n    *   d) `!=`\n    *   **Sua resposta:**\n\n3.  Qual das seguintes opções não é um tipo de dado numérico em Kotlin?\n    *   a) `Int`\n    *   b) `Float`\n    *   c) `Double`\n    *   d) `String`\n    *   **Sua resposta:**\n\n4.  Como você define uma função em Kotlin?\n    *   a) `function myFunction() {}`\n    *   b) `fun myFunction() {}`\n    *   c) `def myFunction() {}`\n    *   d) `void myFunction() {}`\n    *   **Sua resposta:**\n\n5.  Qual é a função usada para imprimir algo no console em Kotlin?\n    *   a) `System.out.println()`\n    *   b) `pr

--------------------------------------------------------------

--- 📝 Resultado do Agente 1 (Buscador) ---



'E aí, futuro mestre do Kotlin! 👋 Prepare-se para uma jornada incrível no mundo da programação, onde você vai criar apps incríveis e resolver problemas como um verdadeiro ninja! Kotlin é uma linguagem de programação moderna, criada para ser fácil de aprender e superpoderosa. Deixa eu te explicar os principais conceitos de um jeito que você nunca mais vai esquecer:\n\n**1. Variáveis: As Caixas Mágicas da Informação** 📦✨\n\nImagine que você tem várias caixas onde pode guardar coisas: brinquedos, livros, cartas... No Kotlin, as variáveis são como essas caixas! Elas servem para guardar informações que o seu programa vai usar.\n\n*   **`var`**: É como uma caixa que você pode abrir e trocar o conteúdo quando quiser. Exemplo:\n\n    ```kotlin\n    var nome = "Alice" // Criei uma caixa chamada \'nome\' e guardei \'Alice\' dentro\n    nome = "Bob"      // Troquei o conteúdo da caixa \'nome\' para \'Bob\'\n    ```\n*   **`val`**: É uma caixa especial, que você só pode preencher uma vez! Depois q

--------------------------------------------------------------
